In [1]:
from ultralytics import YOLO
from ensemble_masks_wbf import *
import albumentations as A
from ensemble_masks_wbf import *
import os
from PIL import Image 
import cv2

In [2]:
hor_flip = A.Compose([
    A.HorizontalFlip(p=1.0)
],bbox_params=A.BboxParams(format='pascal_voc', label_fields=['box_labels']),p=1.0)
    
vert_flip = A.Compose([
    A.VerticalFlip(p=1.0)
],bbox_params=A.BboxParams(format='pascal_voc', label_fields=['box_labels']),p=1.0)

rot = A.Compose([
    A.Rotate(limit=(180,180), p=1.0)
],bbox_params=A.BboxParams(format='pascal_voc', label_fields=['box_labels']),p=1.0)
    
light = A.Compose([
    A.RandomBrightnessContrast(
        p=1,
        brightness_limit=(0.2,0.2),
        contrast_limit=(0,0)
    )
],p=1.0)

inv_transforms = [None, hor_flip, vert_flip, rot, None]

transforms = [
    None,
    A.HorizontalFlip(p=1.0),
    A.VerticalFlip(p=1.0),
    A.Rotate(limit=(180,180), p=1.0),
    light
]


def tta_infer(img_path,model,yolo_kwargs):
    img = cv2.imread(img_path)
    bboxes,masks,scores, = [],[],[]
    
    for tr, inv_tr in zip(transforms,inv_transforms):
        if tr is not None:
            img_tr = tr(image=img)['image']
        else:
            img_tr = img
        
        res = model(img_tr,**yolo_kwargs)[0]
        if res.masks is None:
            continue
        mask = res[0].masks.data.cpu().numpy()
        mask = [cv2.resize(x,(693,1344)).T for x in mask]
        boxes = res[0].boxes.xyxy.cpu().numpy().tolist()
        confs = res[0].boxes.conf.cpu().numpy().tolist()
        if inv_tr is not None:
            res_tr = inv_tr(image=img_tr,masks=mask,bboxes=boxes,box_labels=['box_labels']*len(boxes))
            boxes = res_tr['bboxes']
            mask = res_tr['masks']
        bboxes.append(boxes)
        masks.append(mask)
        scores.append(confs)
    return bboxes,masks,scores

In [3]:
model = YOLO('runs/segment/train2/weights/last.pt')  # Укажите путь к вашей модели

test_folder = "test"  # Папка с тестовыми изображениями
output_folder = "output"  # Папка для сохранения результатов
os.makedirs(output_folder, exist_ok=True)
predictions = []
conf = 0.5
imgsc = (1344,693)

In [ ]:
from tqdm.auto import tqdm, trange
import os
import json
import numpy as np
from ultralytics import YOLO
import albumentations as A
from ensemble_masks_wbf import *
import pycocotools.mask

while conf > 0.01:
    img_files = set([x['image_name'] for x in predictions]) if len(predictions) != 0 else []
    
    for file_name in tqdm(os.listdir(test_folder)):
        image_path = os.path.join(test_folder, file_name)
        if file_name in img_files:
            continue
        image = cv2.imread(image_path)
        bboxes, masks, scores = tta_infer(image_path,model,{'verbose':False,'conf':conf})
        if len(bboxes) == 0:
            continue
        masks = [x[0] for x in masks]
        scores = [x[0] for x in scores]
        bboxes = [x[0] for x in bboxes]
        _, scores, masks, bboxes = nms_predictions(
            classes = [0] * len(bboxes),
            scores = scores,
            bboxes = bboxes,
            masks = masks,
            iou_th = 0.5#0.75
        )
        
        for i in range(len(bboxes)):
            mask = masks[i]
            mask = resize(mask,imgsc)
            rle = pycocotools.mask.encode(np.asfortranarray(mask.astype(np.uint8)))
            segmentation = {
                "size": [693, 1344],
                "counts": rle["counts"].decode("utf-8")
            }

            predictions.append({
                "image_name": file_name,
                "category_id": 1,
                "bbox": bboxes[i].tolist(),
                "score": scores[i],
                "segmentation": segmentation
            })
    conf -= 0.01

# Сохранение предсказаний в JSON
predictions_file = os.path.join(output_folder, "submission_31.json")
with open(predictions_file, "w") as f:
    json.dump(predictions, f, indent=4)

print(f"Обработка завершена. Результаты сохранены в папке: {output_folder}")

  0%|          | 0/9074 [00:00<?, ?it/s]

In [ ]:
mask